In [10]:
import pandas as pd

from IPython.display import HTML
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

cols = ['label'] + [f'I{i}' for i in range(1, 14)] + [f'C{i}' for i in range(1, 27)]

num_features = [f'I{i}' for i in range(1, 14)]  # Numerical features
cat_features = [f'C{i}' for i in range(1, 27)]  # Categorical features

data = pd.read_csv('../../data/raw/train.txt', sep='\t', header=None, names=cols, nrows=10000)

os.makedirs('eda_report_dist', exist_ok=True)

# Get current date in YYYY-MM-DD format
current_date = pd.Timestamp.now().strftime("%Y-%m-%d")
# Define analyst name as variable
analyst_name = "Charle"


In [11]:
# ==================== 1. Label Distribution Analysis ====================
label_dist = data['label'].value_counts(normalize=True).sort_index()
label_counts = data['label'].value_counts().sort_index()

# Generate label distribution table
label_table = pd.DataFrame({
    'Class': ['Negative(0)', 'Positive(1)'],
    'Count': label_counts.values,
    'Percentage(%)': (label_dist.values * 100).round(2)
}).style \
    .format({'Percentage(%)': '{:.2f}%', 'Count': '{:,}'}) \
    .background_gradient(subset=['Percentage(%)'], cmap='Oranges') \
    .to_html()


In [12]:
# ==================== 2. Data Sample Preview ====================
feature_table = (data
                 .head(10)
                 .style
                 .background_gradient(subset=num_features, cmap='Blues')
                 .to_html())

In [13]:
# ========== 3. Missing Value Analysis (Distinguish between numerical and categorical) ====================
total_records = len(data)

# Numerical features missing value statistics
if len(num_features) > 0:
    num_missing_stats = data[num_features].isnull().sum().sort_values(ascending=False)
    num_missing_df = pd.DataFrame({
        'Missing Count': num_missing_stats,
        'Missing Ratio(%)': (num_missing_stats / total_records * 100).round(2)
    })
    num_missing_df = num_missing_df[num_missing_df['Missing Count'] > 0]
else:
    num_missing_df = pd.DataFrame(columns=['Missing Count', 'Missing Ratio(%)'])

# Categorical features missing value statistics
if len(cat_features) > 0:
    cat_missing_stats = data[cat_features].isnull().sum().sort_values(ascending=False)
    cat_missing_df = pd.DataFrame({
        'Missing Count': cat_missing_stats,
        'Missing Ratio(%)': (cat_missing_stats / total_records * 100).round(2)
    })
    cat_missing_df = cat_missing_df[cat_missing_df['Missing Count'] > 0]
else:
    cat_missing_df = pd.DataFrame(columns=['Missing Count', 'Missing Ratio(%)'])


# Generate styled missing value tables (by type)
def highlight_high_missing(val):
    color = 'red' if val > 20 else ''  # Highlight if missing >20% in red
    return f'background-color: {color}' if val > 0 else ''


# Numerical missing value table
num_missing_table = (
    num_missing_df.style
    .map(highlight_high_missing, subset=['Missing Ratio(%)'])
    .format({
        'Missing Count': '{:,}',  # Thousands separator
        'Missing Ratio(%)': '{:.2f}%'  # Percentage format
    })
    .set_table_styles([{
        'selector': 'caption',
        'props': 'font-size: 1.2em; color: #2c3e50; font-weight: bold;'
    }])
    .to_html()
) if not num_missing_df.empty else "<p>No missing values in numerical features</p>"

# Categorical missing value table
cat_missing_table = (
    cat_missing_df.style
    .map(highlight_high_missing, subset=['Missing Ratio(%)'])
    .format({
        'Missing Count': '{:,}',
        'Missing Ratio(%)': '{:.2f}%'
    })
    .set_table_styles([{
        'selector': 'caption',
        'props': 'font-size: 1.2em; color: #2c3e50; font-weight: bold;'
    }])
    .to_html()
) if not cat_missing_df.empty else "<p>No missing values in categorical features</p>"

# Generate visualization charts for missing values by type
num_missing_plot = ""
if not num_missing_df.empty:
    plt.figure(figsize=(12, 8))
    ax = plt.gca()

    # Bar chart (count)
    num_missing_df.head(20)['Missing Count'].plot.bar(
        ax=ax,
        color='#3498db',
        alpha=0.7,
        label='Missing Count'
    )
    ax.set_ylabel('Number of Missing Records', color='#3498db')
    ax.tick_params(axis='y', colors='#3498db')

    # Line chart (ratio)
    ax2 = ax.twinx()
    num_missing_df.head(20)['Missing Ratio(%)'].plot(
        ax=ax2,
        color='#e74c3c',
        marker='o',
        linewidth=2,
        label='Missing Ratio'
    )
    ax2.set_ylabel('Missing Ratio(%)', color='#e74c3c')
    ax2.tick_params(axis='y', colors='#e74c3c')
    ax2.set_ylim(0, 100)

    plt.title(f'Top 20 Missing Values in Numerical Features (Total Records: {total_records:,})', pad=20)
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plot_path = f'eda_report_dist/missing_numerical_plot.png'
    plt.savefig(plot_path)
    plt.close()
    num_missing_plot = '<div style="text-align: center;"><img src="./eda_report_dist/missing_numerical_plot.png" width="900"></div>'

cat_missing_plot = ""
if not cat_missing_df.empty:
    plt.figure(figsize=(12, 8))
    ax = plt.gca()

    # Bar chart (count)
    cat_missing_df.head(20)['Missing Count'].plot.bar(
        ax=ax,
        color='#9b59b6',
        alpha=0.7,
        label='Missing Count'
    )
    ax.set_ylabel('Number of Missing Records', color='#9b59b6')
    ax.tick_params(axis='y', colors='#9b59b6')

    # Line chart (ratio)
    ax2 = ax.twinx()
    cat_missing_df.head(20)['Missing Ratio(%)'].plot(
        ax=ax2,
        color='#e67e22',
        marker='o',
        linewidth=2,
        label='Missing Ratio'
    )
    ax2.set_ylabel('Missing Ratio(%)', color='#e67e22')
    ax2.tick_params(axis='y', colors='#e67e22')
    ax2.set_ylim(0, 100)

    plt.title(f'Top 20 Missing Values in Categorical Features (Total Records: {total_records:,})', pad=20)
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    plot_path = f'eda_report_dist/missing_categorical_plot.png'
    plt.savefig(plot_path)
    plt.close()
    cat_missing_plot = '<div style="text-align: center;"><img src="./eda_report_dist/missing_categorical_plot.png" width="900"></div>'

In [14]:
# ==================== 4. Numerical Feature Discrimination Analysis ====================
def calc_iv_woe(df, feature, target, bins=10, force_cut=False):
    """
    Calculate Information Value (IV) and Weight of Evidence (WOE) for a numerical feature.
    
    Parameters:
    df (pd.DataFrame): Input dataframe containing the feature and target
    feature (str): Name of the numerical feature to analyze
    target (str): Name of the target variable (binary: 0/1)
    bins (int): Number of bins for discretization
    force_cut (bool): If True, use equal-width binning instead of equal-frequency
    
    Returns:
    float: Calculated IV value for the feature
    pd.DataFrame: Dataframe containing WOE and statistics for each bin
    """
    df = df[[feature, target]].copy()

    # Automatic binning (equal frequency binning to avoid empty bins)
    try:
        if not force_cut:
            df['bins'] = pd.qcut(df[feature], q=bins, duplicates='drop')
        else:
            df['bins'] = pd.cut(df[feature], bins=bins)
    except Exception as e:
        print(f"Using equal-width binning instead of equal-frequency binning for {feature}: {str(e)}")
        df['bins'] = pd.cut(df[feature], bins=bins)

    # Calculate bin statistics
    grouped = df.groupby('bins', observed=True).agg({
        target: ['count', 'sum', 'mean']
    })
    grouped.columns = ['total', 'bad', 'bad_rate']
    grouped['good'] = grouped['total'] - grouped['bad']

    # Calculate WOE and IV
    total_good = grouped['good'].sum()
    total_bad = grouped['bad'].sum()

    # Avoid division by zero with small epsilon
    grouped['good_pct'] = grouped['good'] / (total_good + 1e-10)
    grouped['bad_pct'] = grouped['bad'] / (total_bad + 1e-10)
    grouped['woe'] = np.log((grouped['good_pct'] + 1e-10) / (grouped['bad_pct'] + 1e-10))
    grouped['iv'] = (grouped['good_pct'] - grouped['bad_pct']) * grouped['woe']

    iv = grouped['iv'].sum()
    return iv, grouped.reset_index()


# Calculate IV values for all features and create IV value table
iv_values = {}
for col in num_features:
    try:
        iv, woe_df = calc_iv_woe(data, col, 'label', bins=10)
        iv_values[col] = iv
    except Exception as e:
        print(f"Error calculating IV for {col}: {str(e)}")
        iv_values[col] = np.nan

# Create and format IV dataframe
iv_df = pd.DataFrame.from_dict(iv_values, orient='index', columns=['IV_Value']).sort_values('IV_Value', ascending=False)

# Generate styled HTML table for top IV features
iv_table = iv_df.head(20).style \
    .background_gradient(subset=['IV_Value'], cmap='YlOrRd') \
    .format({'IV_Value': '{:.4f}'}) \
    .to_html()

# Generate analysis charts for top numerical features
os.makedirs('eda_report_dist/feature_plots', exist_ok=True)
top_features = iv_df.head(5).index.tolist()  # Take top 5 features with highest IV values

feature_plots_html = ""
for feature in top_features:
    try:
        iv, woe_df = calc_iv_woe(data, feature, 'label', bins=10)

        plt.figure(figsize=(12, 6))

        # Left: Distribution difference of feature across different labels
        plt.subplot(1, 2, 1)
        sns.boxplot(data=data,
                    x='label',
                    y=feature,
                    hue='label',
                    palette=['#3498db', '#e74c3c'],
                    legend=False,
                    showfliers=False)
        plt.title(f'"{feature}" Label Distribution IV={iv:.4f}')
        plt.xlabel('Label')
        plt.ylabel('Feature Value')

        # Right: WOE monotonic trend
        plt.subplot(1, 2, 2)
        plt.plot(woe_df.index, woe_df['woe'],
                 marker='o', color='#2ecc71', linewidth=2)
        plt.axhline(0, color='grey', linestyle='--')
        plt.title(f'"{feature}" WOE Trend')
        plt.xlabel('Bin')
        plt.ylabel('WOE Value')

        # Fix: Explicitly convert bin labels to strings to avoid Matplotlib parsing issues
        bin_labels = [str(bin) for bin in woe_df['bins']]
        # Show only some labels to avoid overcrowding
        step = max(1, len(woe_df) // 5)  # Show at most 5 labels
        plt.xticks(range(0, len(woe_df), step),
                   [bin_labels[i] for i in range(0, len(woe_df), step)],
                   rotation=45, ha='right')  # Rotate labels to avoid overlap

        plt.tight_layout()

        # Save image
        plot_path = f'eda_report_dist/feature_plots/{feature}.png'
        plt.savefig(plot_path, bbox_inches='tight')  # Ensure complete display of labels
        plt.close()

        # Add to HTML report
        feature_plots_html += f"""
        <div class="feature-analysis">
            <h4>{feature} (IV={iv:.4f})</h4>
            <img src="{plot_path}" width="900">
        </div>
        """
    except Exception as e:
        print(f"Error generating plot for {feature}: {str(e)}")

In [15]:
# ==================== 5. Categorical Feature Analysis ====================
def analyze_categorical_features(df, cat_features, top_n=10):
    """
    Analyzes categorical features by calculating cardinality, top categories,
    and generating distribution visualizations.
    
    Parameters:
    df (pd.DataFrame): Input dataframe containing categorical features
    cat_features (list): List of categorical feature names to analyze
    top_n (int): Number of top categories to display
    
    Returns:
    dict: Analysis results containing cardinality and top category information
    """
    analysis_results = {}

    for col in cat_features:
        # Calculate cardinality (number of unique categories)
        cardinality = df[col].nunique()

        # Calculate top N frequent categories
        top_categories = df[col].value_counts().head(top_n)
        top_categories_pct = (top_categories / len(df)).round(4) * 100

        # Store results in dictionary
        analysis_results[col] = {
            'cardinality': cardinality,
            'top_categories': top_categories.index.tolist(),
            'top_counts': top_categories.values.tolist(),
            'top_percentages(%)': top_categories_pct.values.tolist()
        }

        # Generate visualization chart
        plt.figure(figsize=(12, 6))
        ax = sns.barplot(x=top_categories_pct.index,
                         y=top_categories_pct.values,
                         hue=top_categories_pct.index,
                         palette='viridis',
                         legend=False)

        plt.title(f'"{col}" Top {top_n} Categories Distribution\n(Total Cardinality: {cardinality})')
        plt.xlabel('Category')
        plt.ylabel('Percentage (%)')
        plt.xticks(rotation=45)

        # Add value labels on bars
        for p in ax.patches:
            ax.annotate(f'{p.get_height():.1f}%',
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center',
                        xytext=(0, 5),
                        textcoords='offset points')

        plt.tight_layout()
        plt.savefig(f'eda_report_dist/cat_dist_plots/{col}_top{top_n}.png', bbox_inches='tight')
        plt.close()

    return analysis_results


# Execute categorical feature analysis
os.makedirs('eda_report_dist/cat_dist_plots', exist_ok=True)
if len(cat_features) > 0:
    cat_analysis = analyze_categorical_features(data, cat_features)
else:
    cat_analysis = {}


# Generate HTML report for categorical features
def generate_cat_report(cat_analysis):
    """
    Generates HTML report section for categorical feature analysis results.
    
    Parameters:
    cat_analysis (dict): Results from analyze_categorical_features function
    
    Returns:
    str: HTML string containing the categorical feature report
    """
    report_html = ""

    # Create overview table for categorical features
    overview_df = pd.DataFrame({
        'Feature Name': list(cat_analysis.keys()),
        'Cardinality': [x['cardinality'] for x in cat_analysis.values()],
        'Top 1 Category': [x['top_categories'][0] for x in cat_analysis.values()],
        'Top 1 Percentage (%)': [x['top_percentages(%)'][0] for x in cat_analysis.values()]
    })

    overview_table = overview_df.style \
        .background_gradient(subset=['Cardinality'], cmap='YlOrRd') \
        .background_gradient(subset=['Top 1 Percentage (%)'], cmap='Blues') \
        .format({'Top 1 Percentage (%)': '{:.1f}%'}) \
        .to_html()

    report_html += f"""
    <div class="report-section">
        <h2 class="section-title">5. Categorical Features Overview</h2>
        {overview_table}
    </div>
    """

    # Add detailed analysis for each categorical feature
    for col, stats in cat_analysis.items():
        # Create top 10 categories table
        top10_df = pd.DataFrame({
            'Category': stats['top_categories'],
            'Count': stats['top_counts'],
            'Percentage (%)': stats['top_percentages(%)']
        })

        top10_table = top10_df.style \
            .format({'Percentage (%)': '{:.2f}%', 'Count': '{:,}'}) \
            .background_gradient(subset=['Percentage (%)'], cmap='Blues') \
            .to_html()

        report_html += f"""
        <div class="feature-analysis">
            <h3>{col} (Cardinality: {stats['cardinality']})</h3>
            <div class="row" style="height: 300px;"> <!-- Fixed total row height (adjust as needed) -->
                <!-- Table container -->
                <div class="col-md-6 d-flex align-items-stretch"> 
                    <div style="width: 100%; overflow: auto;"> <!-- Scroll when exceeding height -->
                        {top10_table}
                    </div>
                </div>
                <!-- Image container -->
                <div class="col-md-6 d-flex align-items-stretch">
                    <div style="width: 100%; display: flex; align-items: flex-end; justify-content: flex-end;">
                        <img src="eda_report_dist/cat_dist_plots/{col}_top10.png" style="height: 100%; width: auto; object-fit: contain;">
                    </div>
                </div>
            </div>
        </div>
        """

    return report_html


# Generate complete categorical feature report
if cat_analysis:
    cat_report = generate_cat_report(cat_analysis)
else:
    cat_report = "<p>No valid categorical features detected</p>"

In [16]:
# ==================== 6. Numerical Feature Correlation Analysis ====================
# 1. Calculate feature correlation matrix
corr_matrix = data[num_features + ['label']].corr()

# 2. Generate heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Numerical Feature Correlation Heatmap', fontsize=14)
plt.tight_layout()

plt.savefig(f'eda_report_dist/correlation_heatmap.png')
plt.close()

# 3. Generate correlation ranking with target variable
target_corr = corr_matrix['label'].drop('label').sort_values(key=abs, ascending=False)
corr_table = pd.DataFrame({
    'Feature': target_corr.index,
    'Correlation Coefficient': target_corr.values,
    'Correlation Strength': pd.cut(abs(target_corr),
                                   bins=[0, 0.2, 0.4, 0.6, 0.8, 1],
                                   labels=['Very Weak', 'Weak', 'Moderate', 'Strong', 'Very Strong'])
}).head(20)


# 4. Function to detect highly correlated feature pairs
def get_high_corr_pairs(corr_matrix, threshold=0.7):
    """
    Detect highly correlated feature pairs
    Parameters:
        corr_matrix: Correlation coefficient matrix
        threshold: Correlation threshold
    Returns:
        Sorted list of highly correlated feature pairs in format [(feature1, feature2, correlation_coefficient), ...]
    """
    # Create mask for upper triangular matrix
    upper_mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    # Get upper triangular matrix to avoid duplicates
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    high_corr_pairs = []
    # Iterate through all feature pairs
    for col in upper.columns:
        for row in upper.index:
            if upper.loc[row, col] and abs(corr_matrix.loc[row, col]) > threshold:
                high_corr_pairs.append((col, row, corr_matrix.loc[row, col]))
    # Sort by absolute value
    return sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True)


high_corr_pairs = get_high_corr_pairs(corr_matrix.drop('label', axis=1).drop('label', axis=0))

# 5. Generate feature statistics table
feature_stats = pd.DataFrame({
    'Mean': data[num_features].mean(),
    'Standard Deviation': data[num_features].std(),
    'Missing Values Count': data[num_features].isnull().sum(),
    'Missing Values Ratio': data[num_features].isnull().mean(),
    'Minimum': data[num_features].min(),
    '25th Percentile': data[num_features].quantile(0.25),
    'Median': data[num_features].median(),
    '75th Percentile': data[num_features].quantile(0.75),
    'Maximum': data[num_features].max()
}).round(4)

# 6. Generate detailed WOE tables for TOP5 features
top_features = target_corr.head(5).index.tolist()
woe_tables_html = ""

for feature in top_features:
    try:
        iv, woe_df = calc_iv_woe(data, feature, 'label')

        # Enhance WOE table
        woe_df['Sample Count'] = woe_df['total']
        woe_df['Sample Percentage(%)'] = (woe_df['total'] / len(data)) * 100
        woe_df['Bad Rate(%)'] = woe_df['bad_rate'] * 100

        # Format WOE table
        woe_table = woe_df[['bins', 'Sample Count', 'Sample Percentage(%)', 'bad', 'good', 'Bad Rate(%)', 'woe', 'iv']] \
            .rename(columns={
            'bins': 'Bin',
            'bad': 'Bad Count',
            'good': 'Good Count',
            'woe': 'WOE Value',
            'iv': 'IV Contribution'
        }) \
            .style \
            .background_gradient(subset=['Bad Rate(%)'], cmap='Reds') \
            .background_gradient(subset=['WOE Value'], cmap='coolwarm') \
            .format({
            'Sample Percentage(%)': '{:.2f}%',
            'Bad Rate(%)': '{:.2f}%',
            'WOE Value': '{:.4f}',
            'IV Contribution': '{:.6f}'
        }) \
            .set_caption(f'Feature: {feature} (IV={iv:.4f})') \
            .to_html()

        woe_tables_html += f"""
        <div class="woe-analysis">
            <h4>{feature} Binning Analysis (IV={iv:.4f})</h4>
            {woe_table}
        </div>
        """
    except Exception as e:
        print(f"Error generating WOE table for {feature}: {str(e)}")

# 7. Generate correlation analysis HTML content
correlation_html = f"""
<div class="correlation-analysis">
    <h3>Feature Correlation Heatmap</h3>
    <div style="text-align: center;">
        <img src="./eda_report_dist/correlation_heatmap.png" width="800">
    </div>
    
    <h3>Top 20 Features with Strongest Correlation to Target Variable</h3>
    {corr_table.style
.background_gradient(subset=['Correlation Coefficient'], cmap='coolwarm')
.format({'Correlation Coefficient': '{:.4f}'})
.to_html()}
     
    <div class="correlation-classification">
        <strong>Correlation Strength Explanation：</strong><br>
        • 0.0-0.2: Very Weak<br>
        • 0.2-0.4: Weak<br>
        • 0.4-0.6: Moderate<br>
        • 0.6-0.8: Strong<br>
        • 0.8-1.0: Very Strong
    </div>
    
    <h3>Feature Statistics Table</h3>
    {feature_stats.style
.background_gradient(subset=['Missing Values Ratio'], cmap='Reds')
.background_gradient(subset=['Standard Deviation'], cmap='Blues')
.format({
    'Mean': '{:.4f}',
    'Standard Deviation': '{:.4f}',
    'Missing Values Count': '{:,}',
    'Missing Values Ratio': '{:.2%}',
    'Minimum': '{:.4f}',
    '25th Percentile': '{:.4f}',
    'Median': '{:.4f}',
    '75th Percentile': '{:.4f}',
    'Maximum': '{:.4f}'
})
.to_html()}
     
    <h3>TOP5 Features WOE Analysis</h3>
    {woe_tables_html}
</div>
"""

In [21]:
correlation_html += "</div>"

# Combine HTML report
html_report = f"""
<html>
<head>
  <style>
    .report-section {{
      margin: 5px 0;
      padding: 10px;
      border-radius: 5px;
      background: #f9f9f9;
    }}
    .highlight {{
      color: #e74c3c;
      font-weight: bold;
    }}
    .summary {{
      margin: 15px 0;
      padding: 10px;
      background: #ebf5fb;
      border-left: 4px solid #3498db;
    }}
    .iv-classification {{
      background: #f2f4f4;
      padding: 10px;
      margin: 10px 0;
      border-radius: 5px;
    }}
    .feature-analysis {{
      margin-bottom: 30px;
      border-bottom: 1px solid #eee;
      padding-bottom: 20px;
    }}
    .section-title {{
      color: #2c3e50;
      border-bottom: 1px solid #3498db;
      padding-bottom: 5px;
    }}
    .cat-analysis {{
        margin-bottom: 30px;
        border: 1px solid #eee;
        padding: 15px;
        border-radius: 5px;
    }}
    .cat-analysis h3 {{
        color: #2c3e50;
        border-bottom: 1px solid #3498db;
        padding-bottom: 5px;
    }}
    .row {{
        display: flex;
        flex-wrap: wrap;
        margin: 0 -15px;
    }}
    .col-md-6 {{
        flex: 0 0 50%;
        max-width: 50%;
        padding: 0 15px;
        box-sizing: border-box;
    }}
    .correlation-analysis {{
        margin-bottom: 30px;
    }}
    .correlation-analysis ul {{
        list-style-type: none;
        padding-left: 0;
    }}
    .correlation-analysis li {{
        padding: 5px 0;
        border-bottom: 1px solid #eee;
    }}
  </style>
</head>
<body>
  <h1 style="color: #2c3e50; text-align: center;">EDA Report for Feature Engineering</h1>
  <p style="text-align: right; color: #7f8c8d;">Generated: {current_date}    Analyst: {analyst_name}</p>
  
  <div class="report-section">
    <h2 class="section-title">1. Label Distribution Analysis</h2>
    <div class="summary">
      The dataset contains <span class="highlight">{len(data):,}</span> records,
      with positive samples accounting for <span class="highlight">{label_dist[1]:.2%}</span>,
      and negative samples accounting for <span class="highlight">{label_dist[0]:.2%}</span>
    </div>
    {label_table}
  </div>
  
  <div class="report-section">
    <h2 class="section-title">2. Data Sample Preview</h2>
    {feature_table}
  </div>
  
  <div class="report-section">
    <h2 class="section-title">3. Missing Value Analysis</h2>
    <div class="summary">
        A total of <span class="highlight">{len(num_missing_df) + len(cat_missing_df)}</span> features have missing values,<br>
        Numerical features with missing values: <span class="highlight">{len(num_missing_df)}</span>,<br>
        Categorical features with missing values: <span class="highlight">{len(cat_missing_df)}</span>
    </div>
    <h3>Numerical Features Missing Details</h3>
    <div style="display: flex; flex-wrap: wrap; gap: 20px; margin: 20px 0; align-items: center; height: 400px;">
      <!-- Table section - ~40% width -->
      <div style="flex: 0 0 40%; max-width: 40%; height: 100%; overflow: auto; display: flex; align-items: center;">
        <div style="width: 100%;">
          {num_missing_table}
        </div>
      </div>
      <!-- Chart section - ~60% width -->
      <div style="flex: 0 0 55%; max-width: 55%; height: 100%; display: flex; align-items: center; justify-content: center;">
        <div style="max-height: 100%; width: 100%;">
          {num_missing_plot}
        </div>
      </div>
    </div>
    <h3>Categorical Features Missing Details</h3>
    <div style="display: flex; flex-wrap: wrap; gap: 20px; margin: 20px 0; align-items: center; height: 400px;">
      <!-- Table section - ~40% width -->
      <div style="flex: 0 0 40%; max-width: 40%; height: 100%; overflow: auto; display: flex; align-items: center;">
        <div style="width: 100%;">
          {cat_missing_table}
        </div>
      </div>
      <!-- Chart section - ~60% width -->
      <div style="flex: 0 0 55%; max-width: 55%; height: 100%; display: flex; align-items: center; justify-content: center;">
        <div style="max-height: 100%; width: 100%;">
          {cat_missing_plot}
        </div>
      </div>
    </div>
  </div>
  
  <div class="report-section">
    <h2 class="section-title">4. Numerical Feature Discrimination Analysis</h2>
    <div class="summary">
      A total of <span class="highlight">{len(num_features)}</span> numerical features were analyzed,
      the feature with the highest IV value is <span class="highlight">{iv_df.index[0]}</span>(IV={iv_df.iloc[0, 0]:.4f}),
      <span class="highlight">{len(iv_df[iv_df['IV_Value'] > 0.3])}</span> features have strong predictive power (IV>0.3)
    </div>
    <div class="iv-classification">
      <strong>IV Value Predictive Power Classification：</strong><br>
      • < 0.02: No predictive power<br>
      • 0.02-0.1: Weak predictive power<br>
      • 0.1-0.3: Moderate predictive power<br>
      • > 0.3: Strong predictive power
    </div>
    
    <h3>Top 20 Features by IV Value</h3>
    {iv_table}
    
    <h3>Key Features Detailed Analysis</h3>
    {feature_plots_html}
    <p style="color: #7f8c8d;">Note: Complete analysis charts are saved in the feature_plots directory</p>
  </div>
  
  <div class="report-section">
    {cat_report}
  </div>
  
  <div class="report-section">
    <h2 class="section-title">6. Numerical Feature Correlation Analysis</h2>
    <div class="summary">
      The feature with the strongest correlation to the target variable is <span class="highlight">{target_corr.index[0]}</span>(r={target_corr.iloc[0]:.2f}),
      a total of <span class="highlight">{len(target_corr[abs(target_corr) > 0.5])}</span> strongly correlated features were found (|r|>0.5)
    </div>
    {correlation_html}
  </div>
</body>
</html>
"""

HTML(html_report)